This notebook contains the code to generate all figures related to the analysis of the erythroid dataset in the paper. Figures for al other datasets can be generated following the same commands.

All CommFlow's plotting can expoert figures through the using plt.savefig() command, and include the following arguments:
- showfig (default=False): if True, the figure is displayed.
- savefig (default=True): if True, the figure is exported using plt.savefig().
- figname (default=a descriptive name + 'pdf').
- format (default='pdf').

# Import all required packages

In [ ]:
import scvelo as scv
import scanpy as sc
import os
import string
import matplotlib.pyplot as plt

import commflow as cf

This dataset is accessible through scvelo. We download the dataset and run preprocessing and RNA velocity calculation following scvelo's [tutorial](https://scvelo.readthedocs.io/en/stable/index.html).

In [ ]:
adata = scv.datasets.gastrulation_erythroid()              # download the dataset
scv.pp.filter_and_normalize(adata, min_shared_counts=20)   # preprocessing
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)            # compute moments for unspliced/spliced RNA
scv.tl.velocity(adata, mode='stochastic')                  # compute RNA velocity with stochastic model
scv.tl.velocity_graph(adata)                               # compute the velocity graph

# Overview of highly expressed CCC pathways

The cf.tl.pathways_overview command quantifies the strenght of each CCC pathway in the dataset based on either (1) overall expression of ligands and receptors in the pathway or (2) overall number of ligands/receptors that are detected in the pathway. The result of ths overview can be visualized with the cf.pl.pathways_overview command. This analysis is useful to set a cutoff for how many pathways are considered in the follow-up analysis.

In [ ]:
cf.tl.pathways_overview(adata, human=False) #human=False imply using the mouse database
cf.pl.pathways_overview(adata, top=10, ticksize=10, savefig=False, showfig=True)

Run the CCC-based clustering for each pathway with the cf.tl.all_path_sim command. Notable arguments are:
-The second argument is the adata.obs column to use for cell annotations (in this case, 'celltype').
-method is the way to rank CCC pathways.
-n is the number of CCC pathways to consider.
-target: if True, also use downstream targets when estimating CCC pathway expression.
-human: if False, use the mouse database.
-neighbor_average: if True, smoothen ligand/receptor expression based on nearest neighbors.

In [ ]:
cf.tl.all_path_sim(adata, 'celltype', method='expression', n=10, target=False, human=False, neighbor_average=True, print_info=True)

# Update RNA velocity (optional)
In our analysis, we computed RNA velocity with [Unitvelo](https://www.nature.com/articles/s41467-022-34188-7) following [this tutorial](https://unitvelo.readthedocs.io/en/latest/Figure2_ErythroidMouse.html), which yields more robust trajctories for this dataset. The resuklt is stored in a h5ad file "erythroid_lineage_uniTVelo.h5ad" that is loaded here. In the script below, Unitvelo's RNA velocities are transfered to adata.

In [ ]:
adata_uv = sc.read_h5ad('RNA_vel_models/erythroid_lineage_uniTVelo.h5ad')
adata.uns['velocity_graph'] = adata_uv.uns['velocity_graph']
adata.uns['velocity_graph_neg'] = adata_uv.uns['velocity_graph_neg']
adata.uns['velocity_params'] = adata_uv.uns['velocity_params']
adata.uns['label'] = adata_uv.uns['label']
adata.obs['latent_time'] = adata_uv.obs['latent_time']

Compute the mutual information between CCC pathways. It is advisable to run this command right after cf.tl.all_path_sim, even though mutual information analysis will be performed later on in the notebook, because renaming of CCC modes will generate an error when running this command.

In [ ]:
cf.tl.pairwise_MI(adata

# Visualize CCC modes of specific CCC pathways and their distribution along cell lineage

Visualize CCC pathway heterogeneity:

The code below generates an overtview of CCC pathway heterogeneity.
- barplot of pathway heterogeneity.
- Barplot of cell type heterogeneity.
- Heatmap wioth celltype-CCC mode mapping.

In [ ]:
plot_CCC_overview = False
    if plot_CCC_overview:
        cf.pl.pathway_heterogeneity_summary(adata, key='celltype', figname=fig_path + 'pathway_summary.pdf')
        cf.pl.state_heterogeneity_summary(adata, 'celltype', figname=fig_path + 'states_summary.pdf', figsize=(5, 5))

        # shorten cell type names for nicer plotting:
        cf.tl.rename_cells(adata, {'Blood progenitors 1': 'BP-1', 'Blood progenitors 2': 'BP-2', 'Erythroid1': 'Er-1',
                                   'Erythroid2': 'Er-2', 'Erythroid3': 'Er-3'}, key='celltype')
        cf.pl.heterogeneity_heatmap(adata, 'celltype', figname=fig_path + 'het_heatmap.pdf', fontsize=7,
                                    figsize=(2.5, 2.5))

Visualize CCC pathway heterogeneity:

The code below generates 4 different visualization for each CCC pathway that was considered in the CCC_based clustering (in this case, 10 pathways because n=10 in cf.tl.all_path_sim). Visualizations include:
- scatterplot of CCC modes.
- eigenvalue gap.
- cell type heterogeneity with respect to the CCC pathway.
- kdeplot of CCC modes overlayed with RNA velocity trajectories. This figure is exported in png by default.

In [ ]:
plot_CCC_pathway_detail = False
    if plot_CCC_pathway_detail:
        if not os.path.isdir('erythroid/pathway_detail'):
            os.mkdir('erythroid/pathway_detail')
        for p in adata.uns['pathways'].keys():
            cf.pl.scatter2D(adata, p + '_modes', figname=fig_path + 'pathway_detail/' + 'scatter_' + p + '.pdf')
            cf.pl.plot_mode_gap(adata, p, figname=fig_path + 'pathway_detail/' + 'gap_' + p + '.pdf')
            cf.pl.heatmap_one_pathway(adata, p, 'celltype',
                                      figname=fig_path + 'pathway_detail/' + 'table_' + p + '.pdf')
            cf.pl.single_pathway_heterogeneity(adata, p, 'celltype',
                                               figname=fig_path + 'pathway_detail/' + 'heterogeneity_' + p + '.pdf')
            cf.pl.plot_maps(adata, p, figname=fig_path + 'pathway_detail/' + 'sign_prob_' + p + '.png')

# Further composition analysis for specific CCC pathways

We study in more detail the IGF, FGF, and VEGF pathways. First, we rename the CCC modes with more intuotiuve labels (this is done on an ad hoc basis after examinig the results from the previous block of analysis).

In [ ]:
cf.tl.rename_modes(adata, 'IGF', {0: 'Receiver', 1: 'Inactive', 2: 'Sender/Receiver'})
cf.tl.rename_modes(adata, 'FGF', {0: 'Inactive', 1: 'Sender', 2: 'Sender/Receiver'})
cf.tl.rename_modes(adata, 'VEGF', {0: 'Sender', 1: 'Inactive', 2: 'Receiver'})

Viosualize the fraction of cells in different CCC modes as a function of pseudotime:

In [ ]:
igf_frac = cf.pl.sign_prob_plot(adata, 'IGF', key='latent_time', xlab='latent time',
                                        figname=fig_path + 'pseudotime_IGF.pdf', legend_font=8,
                                        title='IGF signaling distribution', showfig=False, return_curve=True)
fgf_frac = cf.pl.sign_prob_plot(adata, 'FGF', key='latent_time', xlab='latent time',
                                        figname=fig_path + 'pseudotime_FGF.pdf', legend_font=8,
                                        title='VEGF signaling distribution', showfig=False, return_curve=True)
vegf_frac = cf.pl.sign_prob_plot(adata, 'VEGF', key='latent_time', xlab='latent time',
                                         figname=fig_path + 'pseudotime_VEGF.pdf', legend_font=8,
                                         title='VEGF signaling distribution', showfig=False, return_curve=True)

Visualize the composition of different CCC modes for the IGF pathway:

In [ ]:
cf.pl.mode_composition(adata, 'celltype', 'IGF', 0, rename_states=['BP-1', 'BP-2', 'Er-1', 'Er-2', 'Er-3'],
                               figname=fig_path + 'pathway_detail/' + 'IGF_receiver.pdf')
cf.pl.mode_composition(adata, 'celltype', 'IGF', 1, rename_states=['BP-1', 'BP-2', 'Er-1', 'Er-2', 'Er-3'],
                               figname=fig_path + 'pathway_detail/' + 'IGF_inactive.pdf')
cf.pl.mode_composition(adata, 'celltype', 'IGF', 2, rename_states=['BP-1', 'BP-2', 'Er-1', 'Er-2', 'Er-3'],
                               figname=fig_path + 'pathway_detail/' + 'IGF_sen_rec.pdf')

# Redundancy and overlap between CCC pathways

In [ ]:
cf.pl.redundancy(adata, figname=fig_path + 'path_redundancy.pdf')

Overlap between CCC modes of different pathways, one pair at a time.

In [ ]:
cf.pl.alluvial_twopath(adata, 'IGF', 'VEGF', key='celltype', figname=fig_path + 'IGF_VEGF.pdf')
cf.pl.alluvial_twopath(adata, 'FGF', 'VEGF', key='celltype', figname=fig_path + 'FGF_VEGF.pdf')
cf.pl.alluvial_twopath(adata, 'IGF', 'FGF', key='celltype', figname=fig_path + 'IGF_FGF.pdf')

# Emerging CCC patterns across pathways

Perform cell clustering based on the CCC mode of the different CCC pathways. The resolution parameter (res) works similarly to standard clustering resolution, i.e., higher resolution will result in more clusters.

In [ ]:
cf.tl.find_sign_patterns(adata, res=0.05)

Summary of the relation between cell tyopes and CCC patterns visualized either as alluvial plot or as barplot.

In [ ]:
cf.tl.pattern_summary(adata, 'celltype', panel_per_row=5, figname=fig_path + 'pattern_summary.pdf')
cf.pl.alluvial_pattern(adata, 'celltype', figname=fig_path + 'pattern_alluvial.pdf')

Detailed visualzation of each CCC pattern:

In [ ]:
if not os.path.isdir('erythroid/pattern_detail'):
    os.mkdir('erythroid/pattern_detail')
for p in set(adata.obs['sign_pattern']):
    cf.tl.pattern(adata, int(p), key='celltype', human=False, ntop=15, figsize=(5, 2), fontsize=7,
                    figname=fig_path + 'pattern_detail/pattern_' + str(p) + '.pdf')
    cf.pl.pattern_plot(adata, pattern=int(p),
                    figname=fig_path + 'pattern_detail/pattern_' + str(p) + '_map.png')
    cf.pl.pattern_prob_plot(adata, int(p), key='latent_time', npoints=10, xlab='latent time', ax=None,
                    figsize=(2, 2), showfig=False, savefig=True,
                    figname=fig_path + 'pattern_detail/pattern_' + str(p) + '_pst.pdf')

# Chord diagram of CCC at meta-cell resolution

For this analysis of the IGF pathway, we imported a set of downstream targets. This gene set is added to the CCC database and will be used to compoute CCC conncetions. This step is optional. We also rename the IGF CCC modes with more intuitive names (this can be skipped if CCC modes were already renamed earlier in the notebook).

In [ ]:
igf = ['CSNK2A1', 'ELK1', 'FOS', 'GRB2', 'HRAS', 'IRS1', 'JUN', 'MAP2K1', 'MAPK3', 'MAPK8', 'PIK3CA', 'PIK3CG',
            'PIK3R1', 'PTPN11', 'RAF1', 'RASA1', 'SHC1', 'SOS1', 'SRF']
igf = [string.capwords(s) for s in igf]
cf.tl.import_database(adata, ['IGF'], species='mouse', unspliced=True, input_target={'IGF': igf})
cf.tl.rename_modes(adata, 'IGF', {0: 'Rec', 1: 'Inac', 2: 'Sen/Rec'})   # rename CCC modes

Compute and visualize the CCC interactions as chord diagram. The chort_diagram() uses R-based chord plotting, so potential errors could be related to proper installation of R.

In [ ]:
cf.tl.compute_ccc_matrix(adata, 'IGF', key='celltype', include_target=True, conversion=True, 
                         model='mass_action', min_cells=10)
cf.pl.chord_diagram(adata, 'IGF', cmap='Set3', key='celltype', figname=fig_path + 'IGF_chord_target', thr=0.01, directional=1)

Visualize the CCC modes and their interactions along the cell lineage. The updat_conn is a dictionary that can be used to update the curvature of CCC arrows for better visualization. 

In [ ]:
update_conn = {'start: BP-1-Sen/Rec; end: Er-2-Rec': "arc3, rad=0.3",
                   'start: BP-1-Sen/Rec; end: Er-1-Rec': "arc3, rad=0.3"}
 cf.pl.coarse_grained_map(adata, 'IGF', key='celltype', update_horiz_align=update_horiz,
                        update_vert_align=update_vert, update_connection=update_conn, thr=0.03,
                        fontsize=7, figsize=(2.5, 2.5),
                        figname=fig_path + 'coarse_grained_IGF.png')